In [1]:
# importing packages
import pandas as pd
import re
import numpy as np

from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
from requests import get
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from time import time
start_time = time()

from warnings import warn

In [2]:
# replace variables here.
# engenharia+mecanica NOT ("seleção+engenharia" OR "divulga+vagas")
url = "https://www.linkedin.com/jobs/search?keywords=engenharia%2Bmecanica%20NOT%20%28%22Sele%C3%A7%C3%A3o%2BEngenharia%22%20OR%20%22Divulga%2BVagas%22%29&location=Brasil&geoId=106057199&trk=public_jobs_jobs-search-bar_search-submit&redirect=false&position=1&pageNum=0"
no_of_jobs = 500

In [3]:
# this will open up new window with the url provided above 
driver = webdriver.Chrome(r'C:\Users\marcellohro\Downloads\chromedriver.exe')
driver.get(url)
sleep(3)
action = ActionChains(driver)

In [4]:
# to show more jobs. Depends on number of jobs selected
i = 2
while i <= (no_of_jobs/25): 
    driver.find_element_by_xpath('/html/body/main/div/section/button').click()
    i = i + 1
    sleep(5)

In [5]:
# parsing the visible webpage
pageSource = driver.page_source
lxml_soup = BeautifulSoup(pageSource, 'lxml')

# searching for all job containers
job_container = lxml_soup.find('ul', class_ = 'jobs-search__results-list')

print('You are scraping information about {} jobs.'.format(len(job_container)))

You are scraping information about 625 jobs.


In [6]:
# setting up list for job information
job_id = []
post_title = []
company_name = []
post_date = []
job_location = []

In [7]:
# for loop for job title, company, id, location and date posted
for job in job_container:
    
    # job title
    job_titles = job.find("span", class_="screen-reader-text").text
    post_title.append(job_titles)
    
    # linkedin job id
    job_ids = job.find('a', href=True)['href']
    job_ids = re.findall(r'(?!-)([0-9]*)(?=\?)',job_ids)[0]
    job_id.append(job_ids)
    
    # company name
    company_names = job.select_one('img')['alt']
    company_name.append(company_names)
    
    # job location
    job_locations = job.find("span", class_="job-result-card__location").text
    job_location.append(job_locations)
    
    # posting date
    post_dates = job.select_one('time')['datetime']
    post_date.append(post_dates)

In [72]:
# setting up list for job information
job_desc = []
level = []
emp_type = []
functions = []
industries = []

In [73]:
# for loop for job description and criterias
for x in range(392,601):
    
    # clicking on different job containers to view information about the job
    job_xpath = '/html/body/main/div/section/ul/li[{}]/img'.format(x)
    driver.find_element_by_xpath(job_xpath).click()
    sleep(2)
    
    # clicking on show more
    try:
        job_xpath = '/html/body/main/section/div[2]/section[2]/div[2]/section/button[1]/icon'
        driver.find_element_by_xpath(job_xpath).click()
    except Exception:
        pass
    try:
        job_xpath = '/html/body/main/section/div[2]/section[2]/div/section/button[1]/icon'
        driver.find_element_by_xpath(job_xpath).click()
    except Exception:
        pass
    sleep(0.5)
        
    # job description
    jobdesc_xpath = '/html/body/main/section/div[2]/section[2]/div'
    job_descs = driver.find_element_by_xpath(jobdesc_xpath).text
    job_desc.append(job_descs)
        
    # job criteria container below the description
    job_criteria_container = lxml_soup.find('ul', class_ = 'job-criteria__list')
    all_job_criterias = job_criteria_container.find_all("span", class_='job-criteria__text job-criteria__text--criteria')
    
    # Seniority level
    seniority_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[1]'
    seniority = driver.find_element_by_xpath(seniority_xpath).text.splitlines(0)[1]
    level.append(seniority)
    
    # Employment type
    type_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[2]'
    employment_type = driver.find_element_by_xpath(type_xpath).text.splitlines(0)[1]
    emp_type.append(employment_type)
    
    # Job function
    function_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[3]'
    job_function = driver.find_element_by_xpath(function_xpath).text.splitlines(0)[1]
    functions.append(job_function)
    
    # Industries
    try:
        industry_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[4]'
        industry_type = driver.find_element_by_xpath(industry_xpath).text.splitlines(0)[1]
        industries.append(industry_type)
    except Exception:
        industries.append(0)

In [21]:
job_data_1_25 = pd.DataFrame({'Job ID': job_id[0:25],
'Date': post_date[0:25],
'Company Name': company_name[0:25],
'Post': post_title[0:25],
'Location': job_location[0:25],
'Description': job_desc,
'Level': level,
'Type': emp_type,
'Function': functions,
'Industry': industries
})

In [30]:
job_data_26_50 = pd.DataFrame({'Job ID': job_id[25:50],
'Date': post_date[25:50],
'Company Name': company_name[25:50],
'Post': post_title[25:50],
'Location': job_location[25:50],
'Description': job_desc,
'Level': level,
'Type': emp_type,
'Function': functions,
'Industry': industries
})

In [57]:
job_data_51_75 = pd.DataFrame({'Job ID': job_id[50:75],
'Date': post_date[50:75],
'Company Name': company_name[50:75],
'Post': post_title[50:75],
'Location': job_location[50:75],
'Description': job_desc,
'Level': level,
'Type': emp_type,
'Function': functions,
'Industry': industries
})

In [62]:
job_data_76_200 = pd.DataFrame({'Job ID': job_id[75:200],
'Date': post_date[75:200],
'Company Name': company_name[75:200],
'Post': post_title[75:200],
'Location': job_location[75:200],
'Description': job_desc,
'Level': level,
'Type': emp_type,
'Function': functions,
'Industry': industries
})

In [69]:
job_data_201_392 = pd.DataFrame({'Job ID': job_id[200:392],
'Date': post_date[200:392],
'Company Name': company_name[200:392],
'Post': post_title[200:392],
'Location': job_location[200:392],
'Description': job_desc,
'Level': level,
'Type': emp_type,
'Function': functions,
'Industry': industries
})

In [83]:
job_data_393_625 = pd.DataFrame({'Job ID': job_id[391:600],
'Date': post_date[391:600],
'Company Name': company_name[391:600],
'Post': post_title[391:600],
'Location': job_location[391:600],
'Description': job_desc,
'Level': level,
'Type': emp_type,
'Function': functions,
'Industry': industries
})

In [86]:
df=job_data_1_25.append(job_data_26_50).append(job_data_51_75).append(job_data_76_200).append(job_data_201_392).append(job_data_393_625)

In [88]:
df.tail()

,Job ID,Date,Company Name,Post,Location,Description,Level,Type,Function,Industry
204,2163163181,2020-09-15,PHD Sistemas de Segurança,Supervisor de Estrutura Naval (Temporário),"Macaé, Rio de Janeiro, Brazil","Realizará atividades de integridade do tanque,...",Mid-Senior level,Full-time,Other,Civil EngineeringStaffing and RecruitingHuman ...
205,2160147317,2020-09-12,Desidy,Projetista,"Canela, Rio Grande do Sul, Brazil",Descrição\nCódigo: 776109\nFunção: Projetista\...,Entry level,Full-time,DesignArt/CreativeInformation Technology,Information Technology and ServicesStaffing an...
206,2157095670,2020-09-09,Margraf Ed. e Industria,Técnico Eletrônico,"Barueri, São Paulo, Brazil",Vaga de Técnico Eletrônico em São Paulo. A com...,Entry level,Full-time,ManagementManufacturing,RestaurantsFood & BeveragesFood Production
207,2019164075,2020-09-03,Desafiorh,Estagio na área de engenharia industrial,"Jaguariúna, São Paulo, Brazil","Cursando Engenharia de Produção, Elétrica, Mec...",Internship,Full-time,EducationTraining,Information Technology and ServicesStaffing an...
208,1830274457,2020-04-19,workcell,Coordenador de Qualidade,"Indaiatuba, São Paulo, Brazil",workcell rh está com 1 vaga(s) em aberto para ...,Entry level,Full-time,Quality Assurance,Mechanical or Industrial EngineeringStaffing a...


In [85]:
# to check if we have all information
print(len(job_id))
print(len(post_date))
print(len(company_name))
print(len(post_title))
print(len(job_location))
print(len(job_desc))
print(len(level))
print(len(emp_type))
print(len(functions))
print(len(industries))

625
625
625
625
625
209
209
209
209
209


In [ ]:
# cleaning description column
job_data['Description'] = job_data['Description'].str.replace('\n',' ')

In [117]:
job_data.head(10)

,Job ID,Date,Company Name,Post,Location,Description,Level,Type,Function,Industry
0,2023055861,2020-09-09,Kalenborn do Brasil,Analista de planejamento,"Vespasiano, Minas Gerais, Brazil",Irá fazer e manter atualizado o cronograma de ...,Associate,Full-time,Business DevelopmentSales,Civil EngineeringStaffing and RecruitingHuman ...
1,1992524613,2020-08-26,Potbelly Sandwich Works,Estagiário de Engenharia,"Belo Horizonte, Minas Gerais, Brazil",Ø Elaboração de relatórios técnicos baseados e...,Not Applicable,Full-time,EducationTraining,ConstructionCivil EngineeringMechanical or Ind...
2,2023679705,2020-09-11,VLI Logística,ANALISTA PROCESSOS ENGENHARIA PLENO - PCM,"Belo Horizonte, Minas Gerais, Brazil",Direct message the job poster from VLI Logísti...,Mid-Senior level,Full-time,Analyst,Logistics and Supply Chain
3,2010325299,2020-08-26,ISQ Brasil,Estagiário de Engenharia,"Belo Horizonte, Minas Gerais, Brazil",Descrição Ø Elaboração de relatórios técnicos...,Internship,Full-time,EducationTraining,Civil EngineeringStaffing and RecruitingHuman ...
4,2011448846,2020-09-08,Gerdau,Especialista – Engenharia de Manutenção – Sist...,"Ouro Branco, Minas Gerais, Brazil","A Gerdau, uma multinacional brasileira present...",Mid-Senior level,Full-time,EngineeringProduction,Mining & Metals
5,2011339278,2020-09-04,Biolab Sanus Farmacêutica,ANALISTA MANUTENÇÃO JR,"São Paulo, Brazil",Direct message the job poster from Biolab Sanu...,Associate,Full-time,SalesBusiness Development,Pharmaceuticals
6,2020397427,2020-09-05,Vale,ENGENHEIRO – CONFIBILIDADE EQUIPAMENTO DE MINA...,"Nova Lima, Minas Gerais, Brazil",Atenção: é necessário anexar seu currículo no ...,Entry level,Full-time,EngineeringInformation Technology,Information Technology and ServicesStaffing an...
7,2023632796,2020-09-10,Adecco,Engenheiro (a) de Planejamento de Projetos,Greater Belo Horizonte,Formação: Superior completo em Engenharia Elé...,Mid-Senior level,Full-time,EngineeringProject Management,Staffing and Recruiting
8,2004846889,2020-09-14,Confidencial,Gerente de Engenharia e Confiabilidade,"São Paulo, Brazil",Direct message the job poster from Confidencia...,Executive,Full-time,EngineeringManagementManufacturing,Consumer GoodsFood ProductionFood & Beverages
9,1989203630,2020-08-18,RHI Magnesita,Projetista,"Contagem, Minas Gerais, Brazil",Carreira morna? Na RHI Magnesita você encontr...,Associate,Full-time,DesignInformation Technology,Mining & Metals


In [89]:
df.to_csv('LinkedIn Job Data.csv', index=0)

In [91]:
imported=pd.read_csv('LinkedIn Job Data.csv')

In [97]:
imported.tail()

,Job ID,Date,Company Name,Post,Location,Description,Level,Type,Function,Industry
596,2163163181,2020-09-15,PHD Sistemas de Segurança,Supervisor de Estrutura Naval (Temporário),"Macaé, Rio de Janeiro, Brazil","Realizará atividades de integridade do tanque,...",Mid-Senior level,Full-time,Other,Civil EngineeringStaffing and RecruitingHuman ...
597,2160147317,2020-09-12,Desidy,Projetista,"Canela, Rio Grande do Sul, Brazil",Descrição\nCódigo: 776109\nFunção: Projetista\...,Entry level,Full-time,DesignArt/CreativeInformation Technology,Information Technology and ServicesStaffing an...
598,2157095670,2020-09-09,Margraf Ed. e Industria,Técnico Eletrônico,"Barueri, São Paulo, Brazil",Vaga de Técnico Eletrônico em São Paulo. A com...,Entry level,Full-time,ManagementManufacturing,RestaurantsFood & BeveragesFood Production
599,2019164075,2020-09-03,Desafiorh,Estagio na área de engenharia industrial,"Jaguariúna, São Paulo, Brazil","Cursando Engenharia de Produção, Elétrica, Mec...",Internship,Full-time,EducationTraining,Information Technology and ServicesStaffing an...
600,1830274457,2020-04-19,workcell,Coordenador de Qualidade,"Indaiatuba, São Paulo, Brazil",workcell rh está com 1 vaga(s) em aberto para ...,Entry level,Full-time,Quality Assurance,Mechanical or Industrial EngineeringStaffing a...
